In [20]:
%pip install pyscisci networkx python-Levenshtein sparse_dot_topn matplotlib boto3 widgetsnbextension ipywidgets "dask[distributed]" --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.49
    Uninstalling botocore-1.34.49:
      Successfully uninstalled botocore-1.34.49
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.49
    Uninstalling boto3-1.34.49:
      Successfully uninstalled boto3-1.34.49

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyscisci.all as pyscisci

import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import dask.dataframe as dd

%matplotlib inline

path2openalex = '/disks/qnap2/data/t-yatima/openalex231124/data/'

DATA_DIR = '/disks/qnap2/data/t-yatima/scisciJP2024_tutorial/'
OUTPUT_WORKS_DF_PKL = DATA_DIR + 'random_works_large_df.pkl'
OUTPUT_WORKS_REL_REF_DF_PKL = DATA_DIR + 'random_rel_ref_large_df.pkl'

myoa = pyscisci.OpenAlex(path2openalex, database_extension='csv.gz', keep_in_memory=True, show_progress=True, enable_dask=True) 

from dask.diagnostics import ProgressBar
ProgressBar().register()

from dask.distributed import progress

In [2]:
all_pub_ddf = myoa.load_publications()
all_pub_ddf

Loading Publications:   0%|          | 0/480 [00:00<?, ?it/s]

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
npartitions=480,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
# 9min
references_df = myoa.load_references()
display(references_df)

Loading pub2ref:   0%|          | 0/480 [00:00<?, ?it/s]

,CitingPublicationId,CitedPublicationId
0,3117235547,55183993
1,3117235547,657382535
2,3117235547,1540739244
3,3117235547,1582266453
4,3117235547,1642177666
...,...,...
9436475,2134809571,2090594024
9436476,2134809571,2124054840
9436477,2134809571,2147932722
9436478,2134809571,2158145285


In [4]:
references_df[references_df['CitingPublicationId'] == 2133895471]

,CitingPublicationId,CitedPublicationId


In [5]:
references_df[references_df['CitedPublicationId'] == 2133895471]

,CitingPublicationId,CitedPublicationId
2382733,2133634407,2133895471
166217,65896575,2133895471
61526,2180197584,2133895471
1910930,2064783168,2133895471
7965554,1510389535,2133895471
8956457,2030085258,2133895471
209772,2012904459,2133895471
3971004,2060282627,2133895471
7427693,1502129278,2133895471
7615629,2074890356,2133895471


In [6]:
# 8min
random_works_ddf = all_pub_ddf.sample(frac=0.002, random_state=1)
random_works_df = random_works_ddf.compute()

[########################################] | 100% Completed | 557.45 s


In [7]:
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
346,3187609235,NaN,2020,0,Des choix contraires à la transition écologiqu...,2020-01-01,article,NaN,NaN,NaN,85,46,46,False,False
237,2266400421,NaN,1989,0,Ballade pour Frédéric. Piano,1989-01-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False
84767,2242094754,NaN,2005,0,Understanding what child protection data mean:...,2005-08-11,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False
183235,2951317825,NaN,2015,0,Desvendando o Universo com o Céu da Serra Gaúcha,2015-11-30,article,NaN,NaN,3,3,NaN,NaN,False,False
198349,3030572269,NaN,2018,0,The estrogen deficiency hypothesis of schizoph...,2018-12-05,article,10.3760/cma.j.issn.1006-7884.2018.06.010,NaN,51,6,385,388,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306631,1698120224,NaN,2002,25,NaN,2002-01-01,article,10.1023/a:1020616324734,NaN,69,3,897,904,False,False
214283,2024816949,NaN,2011,27,A Quality-of-Life Comparison Between Self-Alig...,2011-04-01,article,10.1563/aaid-joi-d-10-00070,20662674,37,sp1,167,173,False,False
321944,2040609328,NaN,1961,25,INFRARED STUDIES ON PYRROLES. THE STRUCTURE OF...,1961-04-01,article,10.1021/j100822a008,NaN,65,4,624,628,False,False
251443,124423791,NaN,1986,26,Paleomagnetism in Arcuate Mountain Belts,1986-01-01,book-chapter,10.1016/b978-0-444-42688-8.50012-6,NaN,NaN,NaN,141,158,False,False


In [8]:
# 11min
all_paa = myoa.load_publicationauthoraffiliation()
all_paa

Loading Publication Author Affiliation:   0%|          | 0/480 [00:00<?, ?it/s]

/home/t-yatima/t-yatima/.venv/lib/python3.10/site-packages/pyscisci/datasource/readwrite.py:203: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat(data)


,PublicationId,AuthorId,AffiliationId,AuthorSequence,OrigAuthorName
0,4287754650,5062226449,NaN,1,Enrico Biffis
1,4287754650,5033140346,NaN,2,Erik Chavez
2,4287754650,5061623148,NaN,3,Alexis Louaas
3,4287754650,5074952038,NaN,4,Pierre M. Picard
4,4287938432,5045351019,NaN,1,Arthur Hugeat
...,...,...,...,...,...
1810902,2134809571,5004197195,1.355989e+08,3,Hiroki Shibata
1810903,2134809571,5061432587,4.210123e+09,4,Tomomi Saito
1810904,2134809571,5029531503,4.240550e+07,5,Osamu Doi
1810905,2134809571,5004358248,4.240550e+07,6,Yutaka Ohya


In [9]:
# all_paaでPublicationIdごとにAuthorIdの数を数える
all_paa.drop(columns=['AffiliationId', 'AuthorSequence', 'OrigAuthorName'], inplace=True)
author_count = all_paa.groupby('PublicationId').count().reset_index()
author_count.columns = ['PublicationId', 'AuthorCount']  # 列名を変更

# random_works_dfに結果を結合する
random_works_df = random_works_df.merge(author_count, on='PublicationId', how='left')
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
0,3187609235,NaN,2020,0,Des choix contraires à la transition écologiqu...,2020-01-01,article,NaN,NaN,NaN,85,46,46,False,False,2.0
1,2266400421,NaN,1989,0,Ballade pour Frédéric. Piano,1989-01-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
2,2242094754,NaN,2005,0,Understanding what child protection data mean:...,2005-08-11,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2.0
3,2951317825,NaN,2015,0,Desvendando o Universo com o Céu da Serra Gaúcha,2015-11-30,article,NaN,NaN,3,3,NaN,NaN,False,False,4.0
4,3030572269,NaN,2018,0,The estrogen deficiency hypothesis of schizoph...,2018-12-05,article,10.3760/cma.j.issn.1006-7884.2018.06.010,NaN,51,6,385,388,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455434,1698120224,NaN,2002,25,NaN,2002-01-01,article,10.1023/a:1020616324734,NaN,69,3,897,904,False,False,3.0
455435,2024816949,NaN,2011,27,A Quality-of-Life Comparison Between Self-Alig...,2011-04-01,article,10.1563/aaid-joi-d-10-00070,20662674,37,sp1,167,173,False,False,4.0
455436,2040609328,NaN,1961,25,INFRARED STUDIES ON PYRROLES. THE STRUCTURE OF...,1961-04-01,article,10.1021/j100822a008,NaN,65,4,624,628,False,False,2.0
455437,124423791,NaN,1986,26,Paleomagnetism in Arcuate Mountain Belts,1986-01-01,book-chapter,10.1016/b978-0-444-42688-8.50012-6,NaN,NaN,NaN,141,158,False,False,2.0


In [10]:
bibmet_id_list = [int(work) for work in random_works_df['PublicationId'].to_list()]
bibmet_id_list

[3187609235,
 2266400421,
 2242094754,
 2951317825,
 3030572269,
 353247501,
 423748994,
 3140467897,
 600221648,
 600340109,
 1129359357,
 806319339,
 2011382101,
 418273658,
 1955343276,
 2261115044,
 593195327,
 2102983573,
 2904163544,
 2254226425,
 3213920623,
 3028340796,
 3197833236,
 333645419,
 2292987227,
 4300175606,
 586206721,
 1021614813,
 2369894610,
 561573473,
 597990456,
 179446277,
 1559185838,
 28721133,
 4318538264,
 4327593038,
 2500416614,
 2379598691,
 2098030745,
 3137758815,
 2269698071,
 2256635544,
 367615199,
 2372627520,
 2313657755,
 2070456871,
 2340998267,
 2283355807,
 75216325,
 3029494918,
 3028650866,
 3088404512,
 958653214,
 162361947,
 969464301,
 2299861510,
 2905853258,
 2921534856,
 641200500,
 2995043144,
 3032579228,
 411410270,
 423560238,
 2261032407,
 1488539424,
 611272427,
 4206311784,
 597022508,
 1929649835,
 573255590,
 75764502,
 4368372884,
 2369687867,
 2742025093,
 2302789741,
 62174780,
 268883542,
 139464679,
 2171183418,
 5721

In [11]:
# 6min
# (B) 引用先の論文の抽出
B_relations = references_df[references_df['CitedPublicationId'].isin(bibmet_id_list)]

# (C) 参考文献に挙げた論文
C_relations = references_df[references_df['CitingPublicationId'].isin(bibmet_id_list)]

# (D) 更に「参考文献に挙げた論文」を参考文献に挙げている論文の抽出
# まず(C)からCitedPublicationIdを取得し、それを引用している論文を探す
C_cited_ids = C_relations['CitedPublicationId'].unique()
D_relations = references_df[references_df['CitedPublicationId'].isin(C_cited_ids)]

B_relations, D_relations

(        CitingPublicationId CitedPublicationId
 95               3117235547         3114353274
 47               2147314816         2147486095
 975              1992508126         1544177623
 57               2099334955         2172011800
 204              2085066400         2137817507
 ...                     ...                ...
 9434797          2134000226         1974362877
 9435026          2134132553         2078226033
 9435067          2134154704         1971449037
 9435815          2134438334         2096531787
 9436043          2134555933         2004159134
 
 [3072521 rows x 2 columns],
         CitingPublicationId CitedPublicationId
 2                3117235547         1540739244
 6                3117235547         1817976406
 8                3117235547         1911154180
 9                3117235547         1938062671
 14               3117235547         2004341664
 ...                     ...                ...
 9436461          2134806910         2142999685
 9436462 

In [12]:
# D_relations[D_relations['CitedPublicationId'] == 1976880180]

In [13]:
# D_relations[D_relations['CitingPublicationId'] == 2573593301]

In [14]:
# references_df[references_df['CitedPublicationId'] == 55183993]

In [15]:
# references_df[references_df['CitedPublicationId'] == '55183993']

In [16]:
works_rel_ref_df = pd.concat([B_relations, C_relations, D_relations]).drop_duplicates().reset_index(drop=True)
works_rel_ref_df.rename(columns={'CitingPublicationId': 'source', 'CitedPublicationId': 'target'}, inplace=True)
works_rel_ref_df

,source,target
0,3117235547,3114353274
1,2147314816,2147486095
2,1992508126,1544177623
3,2099334955,2172011800
4,2085066400,2137817507
...,...,...
431578945,2134806910,2142999685
431578946,2134806910,2143114368
431578947,2134806910,2158058159
431578948,2134806910,4292994367


In [17]:
random_works_df.set_index('PublicationId', inplace=True)
random_works_df

,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
PublicationId,,,,,,,,,,,,,,,
3187609235,NaN,2020,0,Des choix contraires à la transition écologiqu...,2020-01-01,article,NaN,NaN,NaN,85,46,46,False,False,2.0
2266400421,NaN,1989,0,Ballade pour Frédéric. Piano,1989-01-01,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
2242094754,NaN,2005,0,Understanding what child protection data mean:...,2005-08-11,article,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2.0
2951317825,NaN,2015,0,Desvendando o Universo com o Céu da Serra Gaúcha,2015-11-30,article,NaN,NaN,3,3,NaN,NaN,False,False,4.0
3030572269,NaN,2018,0,The estrogen deficiency hypothesis of schizoph...,2018-12-05,article,10.3760/cma.j.issn.1006-7884.2018.06.010,NaN,51,6,385,388,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698120224,NaN,2002,25,NaN,2002-01-01,article,10.1023/a:1020616324734,NaN,69,3,897,904,False,False,3.0
2024816949,NaN,2011,27,A Quality-of-Life Comparison Between Self-Alig...,2011-04-01,article,10.1563/aaid-joi-d-10-00070,20662674,37,sp1,167,173,False,False,4.0
2040609328,NaN,1961,25,INFRARED STUDIES ON PYRROLES. THE STRUCTURE OF...,1961-04-01,article,10.1021/j100822a008,NaN,65,4,624,628,False,False,2.0


In [18]:
random_works_df.to_pickle(OUTPUT_WORKS_DF_PKL)
works_rel_ref_df.to_pickle(OUTPUT_WORKS_REL_REF_DF_PKL)